Note: to run this script, the NSIDC library has been adapted. If you want to download AQUA-AMSRE images, please sure to change the unchecked lines of the def make_measures_url(url_template, res, dt, freq, HV, AD): function. The suffix is different for the DMSP vs. AQUA situation, hence this temporary solution

In [1]:
%reload_ext autoreload
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import xarray as xr
import pyproj
import osr
import datetime
import subprocess
import shutil
import numpy as np

# import our own small lib
import nsidc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
url_base = 'https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/'
url_folder = '{:s}/'
url_template = url_base + url_folder + 'NSIDC-0630-EASE2_T{:s}km-F13_SSMI-{:s}-{:s}{:s}-{:s}-{:s}-v1.3.nc'
# Specify between S(outhern) or N(orthern). T(ropical) is for AMSR-E
#A vs. D AQUA is E vs. M. SSMI
download_template = 'wget --http-user={:s} --http-password={:s} --load-cookies mycookies.txt --save-cookies mycookies.txt --keep-session-cookies --no-check-certificate --auth-no-challenge -r --reject "index.html*" -np -e robots=off {:s}'

fn_out_template = 'NSIDC-0630-EASE2_T{:s}km-F13_SSMI-{:s}{:s}-{:s}_{:04d}.nc'
out_path = os.path.abspath('netcdf')
if not(os.path.isdir(out_path)):
    # prepare the output path
    os.makedirs(out_path)
freq = '37' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
res = '25'  # can also be 12.5 or 25, although this depends on the chosen frequency as well
HV = 'H' # Horizontal or Vertical polarisation
AD = 'A' # MORINING OR EVENING Ascending or descending, descending contains imagery during the day, probably showing more contrast

# start_date = datetime.datetime(2002, 6, 1)
start_date = datetime.datetime(2005, 1, 1)
end_date = datetime.datetime(2010, 1, 1)
# start_date = datetime.datetime(2010, 1, 1) # date after data recovery
#end_date = datetime.datetime(2002, 6, 5)  # only a few days to make a demonstration
# end_date = datetime.datetime(2011, 12, 31)  # use this end date to cover the whole AMSR-E period.

step = datetime.timedelta(days=1)  # M

try:
    credential_fn = os.path.abspath('../../nsidc_credentials.txt')
    username, password = str(np.loadtxt(credential_fn, dtype=np.str)).split(',')
except:
    print('No credential file found, please put a txt file with <user>,<pass> in {:s}'.format(credential_fn))
 

In [3]:
print(' Frequency:',freq,'GHz.', 
      ' Resolution:',res, 'km.',
      ' Polarization:', HV,'.' 
      ' Ascending/descending:', AD)

 Frequency: 37 GHz.  Resolution: 25 km.  Polarization: H . Ascending/descending: A


In [4]:
# define the proj4string belonging to the grids you are downloading. You can find these in a downloaded sample 
# under the variable/attribute crs.proj4text. Here we simply copy-paste that.
proj4str = '+proj=cea +lat_0=0 +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
points_interest = [(2.4406387000000223, 6.893251099999999),
                   (2.2999999999999545, 7.099999999999999),
                   (2.4851462999999967, 8.0335995),
                   (2.0499999999999545, 7.566667)
]

# In this example, we are interested in parts of West-Africa, so we provide a bounding box covering that in the format
# [(xmin, ymin), (xmax, ymax)]
bounds = [(-5., 0.),
          (10., 15.),
         ]


# we define a projection object for lat-lon WGS84 (EPSG code 4326)
proj_out = pyproj.Proj(init='epsg:4326')
# we define a projection object for the projection used in the downloaded grids.
proj_in = pyproj.Proj(proj4str)

# here we convert the coordinates in lat-lon into the coordinate system of the downloaded grids.
bounds_xy = nsidc.proj_coords(bounds, proj_out, proj_in)
points_xy = nsidc.proj_coords(points_interest, proj_out, proj_in)

# now we're all set!

In [ ]:
import time
dt = start_date
list_ds = []
year = dt.year # let's store data per year

while dt <= end_date:
    url = nsidc.make_measures_url(url_template, res, dt, freq, HV, AD)  # make the url string to download
    fn = url.strip('https://')  # strip https:// from the url to get the local location of the downloaded file
    path = fn.split('/')[0]  # split fn in little pieces on / and keep only the 0-th indexed piece (main folder)
    download = nsidc.make_measures_download(download_template, url, username, password)  # pass the url to a download string template
    success = subprocess.call(download)  # call the download string in a command-line (use wget.exe! get it from online)
    if success == 0:
        print('Retrieved {:s}'.format(url))
        # the file was successfully downloaded (if not zero, there was a problem or file is sinmply not available)
        # read file, cut a piece and add it to our list of time steps
        ds = xr.open_dataset(fn, decode_cf=False)
        ds_sel = nsidc.select_bounds(ds, bounds_xy)
        list_ds.append(ds_sel.load())  # load the actual data so that we can delete the original downloaded files
        ds.close()
        #time.sleep(0.001)
        shutil.rmtree(path)  # we're done cutting part of the required grid, so throw away the originally downloaded world grid.
    dt += step   # increase by one day to go for the next download day.
    if (year != dt.year) or (dt > end_date):  # store results if one moves to a new year or the end date is reached
        # concatenate the list of timesteps into a new ds
        if len(list_ds) > 0:
            # only store if any values were found
            ds_year = nsidc.correct_miss_fill(xr.concat(list_ds, dim='time'))
            # ds_year.data_vars
            encoding = {var: {'zlib': True} for var in ds_year.data_vars if var != 'crs'}
            # store the current dataset into a nice netcdf file
            fn_out = os.path.abspath(os.path.join(out_path, fn_out_template.format(str(res), freq, HV, AD, year)))
            print('Writing output for year {:d} to {:s}'.format(year, fn_out))
            ds_year.to_netcdf(fn_out, encoding=encoding)
        # prepare a new dataset
        list_ds = []  # empty list
        year = dt.year  # update the year


Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.01.01/NSIDC-0630-EASE2_T25km-F13_SSMI-2005001-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.01.02/NSIDC-0630-EASE2_T25km-F13_SSMI-2005002-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.01.03/NSIDC-0630-EASE2_T25km-F13_SSMI-2005003-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.01.04/NSIDC-0630-EASE2_T25km-F13_SSMI-2005004-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.01.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2005005-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.01.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2005006-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.01.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2005007-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.03.02/NSIDC-0630-EASE2_T25km-F13_SSMI-2005061-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.03.03/NSIDC-0630-EASE2_T25km-F13_SSMI-2005062-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.03.04/NSIDC-0630-EASE2_T25km-F13_SSMI-2005063-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.03.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2005064-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.03.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2005065-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.03.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2005066-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.03.08/NSIDC-0630-EASE2_T25km-F13_SSMI-2005067-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.05.01/NSIDC-0630-EASE2_T25km-F13_SSMI-2005121-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.05.02/NSIDC-0630-EASE2_T25km-F13_SSMI-2005122-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.05.03/NSIDC-0630-EASE2_T25km-F13_SSMI-2005123-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.05.04/NSIDC-0630-EASE2_T25km-F13_SSMI-2005124-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.05.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2005125-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.05.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2005126-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.05.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2005127-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.06.30/NSIDC-0630-EASE2_T25km-F13_SSMI-2005181-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.07.01/NSIDC-0630-EASE2_T25km-F13_SSMI-2005182-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.07.02/NSIDC-0630-EASE2_T25km-F13_SSMI-2005183-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.07.03/NSIDC-0630-EASE2_T25km-F13_SSMI-2005184-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.07.04/NSIDC-0630-EASE2_T25km-F13_SSMI-2005185-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.07.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2005186-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.07.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2005187-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.08.29/NSIDC-0630-EASE2_T25km-F13_SSMI-2005241-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.08.30/NSIDC-0630-EASE2_T25km-F13_SSMI-2005242-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.08.31/NSIDC-0630-EASE2_T25km-F13_SSMI-2005243-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.09.01/NSIDC-0630-EASE2_T25km-F13_SSMI-2005244-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.09.02/NSIDC-0630-EASE2_T25km-F13_SSMI-2005245-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.09.03/NSIDC-0630-EASE2_T25km-F13_SSMI-2005246-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2005.09.04/NSIDC-0630-EASE2_T25km-F13_SSMI-2005247-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

...

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.02.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2009036-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.02.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2009037-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.02.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2009038-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.02.08/NSIDC-0630-EASE2_T25km-F13_SSMI-2009039-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.02.09/NSIDC-0630-EASE2_T25km-F13_SSMI-2009040-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.02.10/NSIDC-0630-EASE2_T25km-F13_SSMI-2009041-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.02.11/NSIDC-0630-EASE2_T25km-F13_SSMI-2009042-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.04.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2009096-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.04.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2009097-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.04.08/NSIDC-0630-EASE2_T25km-F13_SSMI-2009098-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.04.09/NSIDC-0630-EASE2_T25km-F13_SSMI-2009099-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.04.10/NSIDC-0630-EASE2_T25km-F13_SSMI-2009100-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.04.11/NSIDC-0630-EASE2_T25km-F13_SSMI-2009101-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.04.12/NSIDC-0630-EASE2_T25km-F13_SSMI-2009102-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.06.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2009156-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.06.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2009157-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.06.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2009158-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.06.08/NSIDC-0630-EASE2_T25km-F13_SSMI-2009159-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.06.09/NSIDC-0630-EASE2_T25km-F13_SSMI-2009160-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.06.10/NSIDC-0630-EASE2_T25km-F13_SSMI-2009161-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.06.11/NSIDC-0630-EASE2_T25km-F13_SSMI-2009162-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.08.04/NSIDC-0630-EASE2_T25km-F13_SSMI-2009216-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.08.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2009217-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.08.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2009218-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.08.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2009219-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.08.08/NSIDC-0630-EASE2_T25km-F13_SSMI-2009220-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.08.09/NSIDC-0630-EASE2_T25km-F13_SSMI-2009221-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.08.10/NSIDC-0630-EASE2_T25km-F13_SSMI-2009222-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.10.03/NSIDC-0630-EASE2_T25km-F13_SSMI-2009276-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.10.04/NSIDC-0630-EASE2_T25km-F13_SSMI-2009277-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.10.05/NSIDC-0630-EASE2_T25km-F13_SSMI-2009278-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.10.06/NSIDC-0630-EASE2_T25km-F13_SSMI-2009279-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.10.07/NSIDC-0630-EASE2_T25km-F13_SSMI-2009280-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.10.08/NSIDC-0630-EASE2_T25km-F13_SSMI-2009281-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/2009.10.09/NSIDC-0630-EASE2_T25km-F13_SSMI-2009282-37H-A-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

In [9]:
print(download)

wget --http-user=w.neisingh@gmail.com --http-password=B.en~!N2018 --load-cookies mycookies.txt --save-cookies mycookies.txt --keep-session-cookies --no-check-certificate --auth-no-challenge -r --reject "index.html*" -np -e robots=off https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1997.02.13/NSIDC-0630-EASE2_T25km-F13_SSMI-1997044-37V-A-GRD-RSS-v1.3.nc
